In [5]:
import requests
from bs4 import BeautifulSoup

base_url = "https://www.aeaweb.org"
issues_url = "https://www.aeaweb.org/journals/aer/issues"

response = requests.get(issues_url)
soup = BeautifulSoup(response.content, 'html.parser')

# 获取最新的volume链接
latest_volume_link = soup.find('article', class_='volume-container').find('a')['href']

# 访问最新的volume链接并获取所有文章链接
response = requests.get(base_url + latest_volume_link)
soup = BeautifulSoup(response.content, 'html.parser')
articles = soup.find_all('article', class_='journal-article')

results = []

for article in articles:
    title_tag = article.find('h3', class_='title')
    title = title_tag.text.strip() if title_tag else "N/A"
    
    authors = [author.text.strip() for author in article.find_all('span', class_='family-name')]
    
    # 如果没有作者，跳过这篇文章
    if not authors:
        continue
    
    article_link_tag = article.find('a')
    if not article_link_tag:
        continue
    article_link = article_link_tag['href']
    
    # 访问文章链接并获取摘要
    response = requests.get(base_url + article_link)
    article_soup = BeautifulSoup(response.content, 'html.parser')
    abstract_tag = article_soup.find('section', class_='article-information abstract')
    abstract = abstract_tag.text.strip().replace('Abstract', '') if abstract_tag else "N/A"
    
    results.append({
        'Title': title,
        'Authors': ", ".join(authors),
        'Abstract': abstract
    })

# 保存到文件
with open('D:\\Personal_project\\aer_latest_articles.txt', 'w', encoding='utf-8') as file:
    for result in results:
        file.write("Title: " + result['Title'] + "\n")
        file.write("Authors: " + result['Authors'] + "\n")
        file.write("Abstract: " + result['Abstract'] + "\n\n")

print("Data has been saved to D:\\Personal_project\\aer_latest_articles.txt")



Data has been saved to D:\Personal_project\aer_latest_articles.txt
